# Maryland Poverty Line Credit

Maryland's Poverty Line Credit provides up to 5% of earned income to eligible Maryland filers with income below the poverty line.

## Examples

Because the Poverty Line Credit is based on the poverty line relative to the Earned Income Tax Credit, which is more generous for families with children and which doesn't give more for families with more than three children, it primarily reaches low-income childless filers and filers with more than six children.

In [1]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px


def make(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25, is_tax_unit_head=True)
    members = ["head"]
    if adults > 1:
        sim.add_person(name="spouse", is_tax_unit_head=False)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=5)
        members += [child]
    sim.add_tax_unit(
        name="tax_unit",
        members=members,
    )
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MD")
    sim.vary("employment_income", max=100_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            plc=sim.calc("md_poverty_line_credit")[0].round(),
            plc_mtr=-sim.deriv(
                "md_poverty_line_credit",
                "employment_income",
                wrt_target="head",
            ),
            adults=adults,
            children=children,
        )
    )


l = []
for adults in range(1, 3):
    for children in range(0, 10):
        l.append(make(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    plc="MD Poverty Line Credit",
    plc_mtr="MD Poverty Line Credit marginal tax rate",
    adults="Adults",
    children="Children",
)

fig = px.line(
    df,
    "employment_income",
    "plc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Maryland Poverty Line Credit",
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, df.plc.max() * 1.05],
)
fig.show()

Maryland's Poverty Line Credit creates marginal tax rates as low as -15.3%, and then infinite marginal tax rates at its ineligibility cliff (100% of the poverty line).

In [3]:
fig = px.line(
    df,
    "employment_income",
    "plc_mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Maryland Poverty line credit marginal tax rate",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".1%", yaxis_range=[-0.2, 0.2]
)
fig.show()

## Budgetary impact

Applying 2022 rules to the 2020 Current Population Survey, OpenFisca US estimates that the Maryland Poverty Line Credit costs \$17 million.

In [4]:
from policyengine_us import Microsimulation

sim = Microsimulation(year=2020)

sim.calc("md_poverty_line_credit", period=2022).sum() / 1e6

17.009740359952332